In [1]:
import OpenAIGymAPI as api
import numpy as np
import scipy as sp
import spike
import gym

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [8]:
pip uninstall clr

^C
Note: you may need to restart the kernel to use updated packages.


## Generate Dummy Data

In [2]:
time, activity, abs_activity, median_abs_deviations, channel_MAD = spike.start_to_finish()

In [3]:
len(time)

1024

In [4]:
activity

array([[ 0.00023815,  0.00235241, -0.00502757, ..., -0.00327567,
         0.00271145, -0.0022081 ],
       [-0.05169021,  0.05628628, -0.06095975, ..., -0.00178051,
         0.00143683, -0.00114067],
       [-0.02760121,  0.0321584 , -0.03679477, ..., -0.00620835,
         0.00510486, -0.00412974],
       ...,
       [-0.03728466,  0.03828681, -0.03913621, ..., -0.00346757,
         0.00288019, -0.00235305],
       [ 0.03777691, -0.03582876,  0.03383159, ..., -0.00372548,
         0.00301838, -0.00240676],
       [-0.04876603,  0.05178601, -0.0547176 , ...,  0.00649217,
        -0.00529067,  0.00424387]])

In [5]:
channel_MAD

array([[0.00019793, 0.00268347, 0.00522166, ..., 0.00329096, 0.00274093,
        0.00222696],
       [0.05163364, 0.05631437, 0.06101586, ..., 0.00178632, 0.00145439,
        0.00115435],
       [0.02754613, 0.03218924, 0.03685292, ..., 0.00623653, 0.00516321,
        0.00416861],
       ...,
       [0.03729751, 0.03819594, 0.03903773, ..., 0.0034858 , 0.00291219,
        0.00237236],
       [0.03781724, 0.03581239, 0.03378645, ..., 0.00373531, 0.00305217,
        0.00243399],
       [0.0487602 , 0.05173462, 0.0546673 , ..., 0.00650992, 0.00534717,
        0.00428836]])

In [6]:
def get_action_from_dummy_data(abs_activity, median_abs_deviations):
    """
    Use the dummy neural activity to decide an action for CartPole.
    """
    # Aggregate spike activity across channels (e.g., summing or averaging)
    # spike_counts = spike.count_spikes(abs_activity, median_abs_deviations)
    spike_counts = np.sum(abs_activity, axis=1)
    # Use only the first two channels to decide the action
    logits = spike_counts[:2]  # Extract the first two channels
    probabilities = sp.special.softmax(logits)  # Convert to probabilities using softmax
    action = np.argmax(probabilities)  # Choose the action with the highest probability
    
    return action, probabilities

In [7]:
abs_activity

array([[0.00023815, 0.00235241, 0.00502757, ..., 0.00327567, 0.00271145,
        0.0022081 ],
       [0.05169021, 0.05628628, 0.06095975, ..., 0.00178051, 0.00143683,
        0.00114067],
       [0.02760121, 0.0321584 , 0.03679477, ..., 0.00620835, 0.00510486,
        0.00412974],
       ...,
       [0.03728466, 0.03828681, 0.03913621, ..., 0.00346757, 0.00288019,
        0.00235305],
       [0.03777691, 0.03582876, 0.03383159, ..., 0.00372548, 0.00301838,
        0.00240676],
       [0.04876603, 0.05178601, 0.0547176 , ..., 0.00649217, 0.00529067,
        0.00424387]])

In [8]:
action, probabilities = get_action_from_dummy_data(abs_activity, median_abs_deviations)
probabilities

array([6.39534719e-07, 9.99999360e-01])

In [9]:
episodes = 100
scores = []
env = gym.make('CartPole-v1', render_mode="human") 
env = env.unwrapped

# Modify the angle threshold for termination (default is ~0.209 radians or 12 degrees)
env.theta_threshold_radians = 0.524 # Approximately amount corresponding to 30 degrees

for e in range(episodes):  
    state, _ = env.reset()
    done, score = False, 0
    while not done:
        # Obtain action based on your spike processing function
        time, activity, abs_activity, median_abs_deviations, channel_MAD = spike.start_to_finish()
        action, probabilities = get_action_from_dummy_data(abs_activity, median_abs_deviations)
        
        # Take a step in the environment
        next_state, reward, done, _, _ = env.step(action)
        score += reward
        
        if done:
            scores.append(score)
            if not e % 10:
                print(f"episode: {e}/{episodes}, score: {score}")
                
        print(f"State: {state}")
        print(f"Action Taken: {action} ('0' = Left, '1' = Right)")
        print(f"Next State: {next_state}")
        print(f"Reward: {reward}")
        print(f"Done: {done}")


2025-03-13 18:16:07.705 Python[38472:2812654] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


State: [ 0.02727038  0.0102057  -0.02041868 -0.03960016]
Action Taken: 0 ('0' = Left, '1' = Right)
Next State: [ 0.02747449 -0.1846176  -0.02121068  0.24657118]
Reward: 1.0
Done: False
State: [ 0.02727038  0.0102057  -0.02041868 -0.03960016]
Action Taken: 0 ('0' = Left, '1' = Right)
Next State: [ 0.02378214 -0.3794303  -0.01627926  0.532489  ]
Reward: 1.0
Done: False
State: [ 0.02727038  0.0102057  -0.02041868 -0.03960016]
Action Taken: 0 ('0' = Left, '1' = Right)
Next State: [ 0.01619354 -0.57431954 -0.00562948  0.81999826]
Reward: 1.0
Done: False
State: [ 0.02727038  0.0102057  -0.02041868 -0.03960016]
Action Taken: 1 ('0' = Left, '1' = Right)
Next State: [ 0.00470715 -0.379121    0.01077049  0.52555007]
Reward: 1.0
Done: False
State: [ 0.02727038  0.0102057  -0.02041868 -0.03960016]
Action Taken: 0 ('0' = Left, '1' = Right)
Next State: [-0.00287527 -0.57439286  0.02128149  0.8216073 ]
Reward: 1.0
Done: False
State: [ 0.02727038  0.0102057  -0.02041868 -0.03960016]
Action Taken: 0 ('

In [10]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.
